## 9. Lifetime and Memory

* Memory Safety is a key selling point of Rust.
* Memory Safety is ensured by very tight controls about who can use what and where.
* __Ownership__ and __Borrowing__ are unique features to Rust that guarantee memory safety without garbage collectors.

### 9.1 Ownership

What is Ownership? [(source)](https://doc.rust-lang.org/book/ch04-01-what-is-ownership.html)

* a set of rules that govern how memory is managed
* all programs use a computer’s memory while running
* common apporaches to memory management
* 1: some languages have garbage collection that looks for no-longer-used memory as the program runs
* 2: in other languages, the programmer must explicitly allocate and free the memory
  * usually with keywords like `malloc` and `free`
* 3: Rust has a system of ownership with a set of rules that the compiler checks
  * if any of the rules are violated, the program won’t compile

Simple Example
* the variable `v` owns the data associated with it
  * `let v = vec![1, 2, 3]; `
* `v` is on the stack while the data is on the heap
* only one variable can have ownership of data / memory used in the heap
  * this avoids _race conditions_ during run-time
* if the data gets re-assigned the ownership is transfered or "moved" 
  * `let v2 = v;`
* trying to access the data / heap through the old variable will cause an error
  * `[E0382] Error: borrow of moved value: 'v'`

<p align="center"><img src="ownership.png" style="width:600px;height400px;"></p>

* this kind of copying will work primitiv types like integers, floats, chars, & booleans

Re-assignment
* if re-assignment of primitive type, it is a COPY operation
  * to put a primitive on the heap, put it in a `Box::new()`
* if re-assignment is NOT a primitive,
  * then by default it is a MOVE operation
  * unless the data type implements the COPY trait  

In [2]:
// "v" owns the data associated with it; "v" is on the stack while the data is on the heap
let v = vec![1, 2, 3]; 
// only one variable can have ownership over the data
let v2 = v;
// [E0382] Error: borrow of moved value: `v`
//println!("{:?}", v)

### 9.2 Borrowing

Intro
* __ownership__ is one main feature that guarantees memory safety in Rust
* to extend operability on variables and maintain memory safety, __borrowing__ is another key feature

What is Borrowing? [(source)](https://doc.rust-lang.org/book/ch04-02-references-and-borrowing.html)
* if a variable is passed to a function or closure, then it is consumed (used  and taken out of memory)
* while it is possible to return variables from functions / closures, _borrowing_ is the better approach
* borrowing is done by providing a _reference_ to variable
* a reference `&` is 
  * like a pointer, as it can follow the memory address and access the data
  * unlike a pointer, because it guarantees a valid values for life of the reference
* accessing the referenced data is done with `*` before the 
  * there can be multiple references to a resource
  * BUT only one mutable reference (prevent race conditions)
  * thus, the borrow has to match the im/mutability of the original
  

In [3]:
let mut a = 40;
{
    let b = &mut a; // b is a mutable reference to a
    *b -= 2;  // data can be accessed by * 
}
// b only exists closure and its referencing/accessing ends with the clorsure
// error: if b would not relinquish control over a 
println!("a = {}", a);

a = 38


In [4]:
fn return_demo(){
// closure function with return
    let print_vec = |x:Vec<i32>| -> Vec<i32>{
        println!("vec={:?}", x);
        return x
    };

    let v = vec![3, 4, 5];
    print_vec(v);
}
return_demo()

vec=[3, 4, 5]


()

In [5]:
fn borrow_demo(){
// closure function with reference
    let print_vec = |x:&Vec<i32>|{
        println!("vec={:?}", x);
    };

    let v = vec![3, 4, 5];
    print_vec(&v);
}

borrow_demo()

vec=[3, 4, 5]


()

### 9.3 Lifetime

* __lifetime__ defines how long a variable will live during execution
  * the lifetime is given to a variable by an `'`
  * `'static` means the variables lives as long as the program itself
  * `<'z>` can be used to build a generic lifetime
* example of why this is useful
  * if one struct `Company` calls on another struct `Person`, then `Person` must exist as long as `Company` exists in the code
  * Rust will not let you compile code where `Person` would disappear before `Company` 
  * if you try, you'll get an error `[E0106] Error: missing lifetime specifier`

<p align="center"><img src="lifetime_error.png" style="width:400px;heigh300px;"></p>


In [6]:
use std::fmt::Debug;

#[derive(Debug)]
struct Person{ name: String }

#[derive(Debug)]
struct Company<'z>{
    name: String,
    ceo: &'z Person
}

fn lifetime_demo(){
    let boss = Person{name: String::from("Elon Musk")};
    let tesla = Company{name: String::from("Tesla"), ceo: &boss};
    println!("Company = {:?}", tesla)
}

lifetime_demo()

Company = Company { name: "Tesla", ceo: Person { name: "Elon Musk" } }


()

### 9.4 Lifetime in Structure Implementation

* using variable lifetimes with generics and passing them to functions
* a `str` type (=string slice) always needs a lifetime
* in order to use variable with certain lifetimes from `struct` in an `impl`ementation, it also needs to "inherit" the same lifetime

In [7]:
struct Human<'a>
{
    name: &'a str
}

impl<'a> Human<'a>
{
    fn talk(&self){
        println!("Hi, my name is {}.", self.name);
    }
}


let h = Human{name: "Tom"};
h.talk();

Hi, my name is Tom.


### 9.5 Reference-Counted Variables (Rc)

* the guarantees for memory safety require a Rust developer to always consider ownership and borrowing
* but there is another way of handing around variables in your code
* the standard library `std::rc:Rc` helps that
  * it keeps count (under the hood) where a variable is referenced
  * when the counter goes down to zero, then variable is removed 

In [8]:
use std::rc::Rc;

let obj0 = Rc::new("Object".to_string());
let obj1 = obj0.clone();
let obj2 = obj0.clone();
println!("Num of Strong pointers to my 'obj' = {}", Rc::strong_count(&obj0))

Num of Strong pointers to my 'obj' = 3


()

In [9]:
use std::rc::Rc;

#[derive(Debug)]
struct Person{ name: String }

impl Person
{
    fn new(name: String) -> Person{
        Person {name: name}
    }
    fn greet(&self){
        println!("Hi, my name is {}.", self.name)
    }
}

fn rc_demo()
{
    let name = Rc::new("Tom".to_string());
    let person = Person::new(name.to_string().clone());
    person.greet();
    println!("name: {}", name.clone())
}

rc_demo()

Hi, my name is Tom.
name: Tom


()

### 9.6 Atomic Reference-Counted Variables (Arc)

* `Rc` variables are not safe to pass between different threads 
  * because parallel threads reference the same data in memory is not safe
* to safely pass variables between threads use _atomic reference counted_ from `std::sync::Arc`
  * it keeps count (under the hood) and queues where and where a variable used
  * `Arc` does not protect against concurrent access however (see next section)

In [10]:
use std::sync::Arc;
use std::thread;

fn arc_demo()
{
    let name = Arc::new("Tom".to_string());
    let person = Person::new(name.to_string().clone());

    let t = thread::spawn(move || {
        person.greet();
    }
    );
    
    println!("name: {}", name.clone());
    t.join().unwrap();
}

arc_demo()

name: Tom
Hi, my name is Tom.


()

### 9.7 Using a Mutex for Thread-Safety Mutability

* `Arc` does not protect against concurrent access through multiple threads
* keyword `mutex` means mutually exclusive and prevents threads from simultaneously modifying the data in memory 
  * only one at a time can do it _.lock()_ing the variable to the thread 

In [11]:
use std::sync::Mutex;

fn mutex_num() {
    let my_mutex = Mutex::new(5); // A new Mutex<i32>. We don't need to say mut
    let mut mutex_changer = my_mutex.lock().unwrap(); // mutex_changer is a MutexGuard
                                                     // It has to be mut because we will change it
                                                     // Now it has access to the Mutex
                                                     // Let's print my_mutex to see:

    println!("0: {:?}", my_mutex); // This prints "Mutex { data: <locked> }"
                                // So we can't access the data with my_mutex now,
                                // only with mutex_changer

    println!("1: {:?}", mutex_changer); // This prints 5. Let's change it to 6.

    *mutex_changer = 6; // mutex_changer is a MutexGuard<i32> so we use * to change the i32

    println!("2: {:?}", mutex_changer); // Now it says 6
}

mutex_num()

0: Mutex { data: <locked>, poisoned: false, .. }
1: 5
2: 6


()

In [ ]:
use std::sync::{Arc, Mutex};

#[derive(Debug)]
struct Person{
    name: Arc<String>,
    state: Arc<Mutex<String>>
}

impl Person
{
    fn new(name: Arc<String>, state: Arc<Mutex<String>>) -> Person{
        Person {name: name, state: state}
    }
    fn greet(&self){
        let mut state = self.state.lock().unwrap();
        state.clear();
        state.push_str("excited");

        println!("Hi, my name is {}. I feel {}", self.name, state.as_str())        
    }
}


fn mutex_demo()
{
    let name = Arc::new("Tom".to_string());
    let state = Arc::new(Mutex::new("bored".to_string()));
    let person = Person::new(name.clone(), state.clone());

    let t = thread::spawn(move || {
        person.greet();
    }
    );
    
    println!("name: {}, state: {}", name.clone(), state.lock().unwrap());
    t.join().unwrap();
}

mutex_demo()